# Validación de códigos IATA en vuelos ✈️

Este notebook valida la **consistencia** entre `flights.csv` (vuelos), `airlines.csv` (aerolíneas) y `airports.csv` (aeropuertos):

- Normaliza texto (trim + mayúsculas) para evitar falsos inválidos.
- Calcula porcentajes de **códigos inválidos** (aerolínea, origen, destino).
- Genera un **gráfico de barras** con los porcentajes.
- Exporta los **registros inválidos** y **tops de códigos inválidos** a CSV.

> **Tip:** Ejecuta cada celda en orden. Ajusta la ruta base donde tienes tus CSV en la siguiente celda.

In [ ]:
# 📁 Rutas de entrada/salida (ajusta según tu entorno)
import os

# Ruta donde tienes los CSV (ajústala). Ejemplo tomado de tu proyecto:
ruta_excel = r"D:/OneDrive/DOCUMENTOS/Personales/2024/uniandes/8 S/seminario/g11-caso-estudio-complexivo/data/"

ruta_vuelos      = os.path.join(ruta_excel, "flights.csv")
ruta_aerolineas  = os.path.join(ruta_excel, "airlines.csv")
ruta_aeropuertos = os.path.join(ruta_excel, "airports.csv")

# Carpeta de salidas
outdir = os.path.join(os.getcwd(), "salidas_validacion")
os.makedirs(outdir, exist_ok=True)
print("➡️  flights:", ruta_vuelos)
print("➡️  airlines:", ruta_aerolineas)
print("➡️  airports:", ruta_aeropuertos)
print("📂  outdir:", outdir)

In [ ]:
# 📦 Carga de datos
import pandas as pd
import matplotlib.pyplot as plt

pd.options.display.float_format = "{:,.3f}".format

vuelos = pd.read_csv(ruta_vuelos, low_memory=False)
aerolineas = pd.read_csv(ruta_aerolineas, low_memory=False)
aeropuertos = pd.read_csv(ruta_aeropuertos, low_memory=False)

print("vuelos:", vuelos.shape)
print("aerolineas:", aerolineas.shape)
print("aeropuertos:", aeropuertos.shape)

display(vuelos.head(3))
display(aerolineas.head(3))
display(aeropuertos.head(3))

In [ ]:
# 🧼 Normalización de claves IATA y columnas relacionadas
def normaliza_col(df: pd.DataFrame, col: str) -> None:
    df[col] = df[col].astype(str).str.strip().str.upper()

for df, col in [
    (vuelos, "AIRLINE"),
    (vuelos, "ORIGIN_AIRPORT"),
    (vuelos, "DESTINATION_AIRPORT"),
    (aerolineas, "IATA_CODE"),
    (aeropuertos, "IATA_CODE"),
]:
    normaliza_col(df, col)

print("Normalización completada ✔️")

In [ ]:
# ✅ Conjuntos válidos y máscaras de inválidos
set_aerolineas  = set(aerolineas["IATA_CODE"].dropna())
set_aeropuertos = set(aeropuertos["IATA_CODE"].dropna())

mask_aerolinea_invalida = ~vuelos["AIRLINE"].isin(set_aerolineas)
mask_origen_invalido    = ~vuelos["ORIGIN_AIRPORT"].isin(set_aeropuertos)
mask_destino_invalido   = ~vuelos["DESTINATION_AIRPORT"].isin(set_aeropuertos)
mask_total_invalido     = mask_aerolinea_invalida | mask_origen_invalido | mask_destino_invalido

porc_aerolinea = mask_aerolinea_invalida.mean() * 100
porc_origen    = mask_origen_invalido.mean() * 100
porc_destino   = mask_destino_invalido.mean() * 100
porc_total     = mask_total_invalido.mean() * 100

print("📊 Porcentaje de registros con códigos inválidos:")
print(f"Aerolíneas no válidas: {porc_aerolinea:.3f}%")
print(f"Aeropuertos de ORIGEN no válidos: {porc_origen:.3f}%")
print(f"Aeropuertos de DESTINO no válidos: {porc_destino:.3f}%")
print(f"🚫 Registros totales con datos inválidos (aerolinea/origen/destino): {porc_total:.3f}%")

In [ ]:
# 📈 Gráfico de barras (matplotlib, un solo gráfico, sin estilos ni colores especificados)
etiquetas = ["Aerolínea inválida", "Origen inválido", "Destino inválido", "Cualquier inválido"]
valores = [porc_aerolinea, porc_origen, porc_destino, porc_total]

plt.figure()
plt.bar(etiquetas, valores)
plt.ylabel("Porcentaje (%)")
plt.title("Porcentaje de registros con códigos inválidos")
plt.xticks(rotation=15)
plt.tight_layout()
chart_path = os.path.join(outdir, "porcentajes_invalidos.png")
plt.savefig(chart_path, dpi=150)
print("Gráfico guardado en:", chart_path)
plt.show()

In [ ]:
# 💾 Exportar filas inválidas a CSV
invalid_dir = os.path.join(outdir, "invalidos")
os.makedirs(invalid_dir, exist_ok=True)

vuelos.loc[mask_aerolinea_invalida].to_csv(os.path.join(invalid_dir, "vuelos_aerolinea_invalida.csv"), index=False)
vuelos.loc[mask_origen_invalido].to_csv(os.path.join(invalid_dir, "vuelos_origen_invalido.csv"), index=False)
vuelos.loc[mask_destino_invalido].to_csv(os.path.join(invalid_dir, "vuelos_destino_invalido.csv"), index=False)
vuelos.loc[mask_total_invalido].to_csv(os.path.join(invalid_dir, "vuelos_cualquier_invalido.csv"), index=False)

print("CSV de inválidos guardados en:", invalid_dir)

In [ ]:
# 🔝 TOP códigos inválidos más frecuentes
def top_invalids(series: pd.Series, valid_set: set, topn: int = 15) -> pd.DataFrame:
    s = series[~series.isin(valid_set)]
    return s.value_counts().head(topn).rename_axis("codigo").reset_index(name="frecuencia")

top_airline = top_invalids(vuelos["AIRLINE"], set_aerolineas)
top_origen  = top_invalids(vuelos["ORIGIN_AIRPORT"], set_aeropuertos)
top_destino = top_invalids(vuelos["DESTINATION_AIRPORT"], set_aeropuertos)

display(top_airline)
display(top_origen)
display(top_destino)

top_dir = os.path.join(outdir, "tops")
os.makedirs(top_dir, exist_ok=True)
top_airline.to_csv(os.path.join(top_dir, "top_aerolineas_invalidas.csv"), index=False)
top_origen.to_csv(os.path.join(top_dir, "top_origenes_invalidos.csv"), index=False)
top_destino.to_csv(os.path.join(top_dir, "top_destinos_invalidos.csv"), index=False)
print("TOPs guardados en:", top_dir)

## Próximos pasos sugeridos
- Mapear y **corregir** códigos comunes inválidos (ej.: espacios internos, alias históricos).
- **Eliminar** filas con claves inexistentes antes de `merge`.
- Registrar estos hallazgos en tu **pipeline de limpieza** para que sea reproducible.